# The Hangman Game
### Using Tensorflow version 0.10. Latest version is 1.2. Some functions differ slightly in the latest version.

In [2]:
import tensorflow as tf
import numpy as np

In [13]:
# allowing better python 2 & python 3 compatibility 
from __future__ import print_function 

def hangman(secret_word, guesser, max_mistakes=8, verbose=True):
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            try:
                guessed.add(guess)
            except:
                print(guessed,guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

def human(mask, guessed):
    print('Enter your guess:')
    return raw_input().lower().strip()
    #return input().lower().strip() # swap with above for python 3


In [14]:
hangman('whatever', human, 8, True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ length 8
You have 8 attempts remaining.
Enter your guess:
fffffff
Guess is fffffff
Sorry, try again.
You have 7 attempts remaining.
Enter your guess:
f
Guess is f
Sorry, try again.
You have 6 attempts remaining.
Enter your guess:
f
Guess is f
Already guessed this before.
You have 5 attempts remaining.
Enter your guess:
f
Guess is f
Already guessed this before.
You have 4 attempts remaining.
Enter your guess:
f
Guess is f
Already guessed this before.
You have 3 attempts remaining.
Enter your guess:
f
Guess is f
Already guessed this before.
You have 2 attempts remaining.
Enter your guess:
f
Guess is f
Already guessed this before.
You have 1 attempts remaining.
Enter your guess:
f
Guess is f
Already guessed this before.
Out of guesses. The word was whatever


8

## Loading the dataset

In [5]:
f = open('corncob_lowercase.txt').read().splitlines()

# Randomly permutes the words
f = [f[i] for i in np.random.permutation(len(f))]
print f[:10]
max_len = np.max([len(x) for x in f])
print("Max len:", max_len)

['depopulated', 'reverentially', 'legends', 'firebrand', 'guardedness', 'ravishes', 'interred', 'dividers', 'droopy', 'disagreed']
('Max len:', 22)


In [9]:
test = []
train = []
for i,item in enumerate(f):
    if i%58 == 0 and len(test) < 1000:
        test.append(item)
    else:
        train.append(item)

len(train), len(test)

(57110, 1000)

# Creating the Neural Network

In [7]:
initializer = tf.contrib.layers.xavier_initializer()
lstm_dim = 300
model_weight_file = "model.weights"
load_model = False
batch_size = 64

In [25]:
def fclayer(inp, num_out, act=tf.nn.tanh, name=None, reuse=None):
    if name is None:
        name = "fclinear"
    with tf.variable_scope(name, reuse=reuse):
        shape = inp.get_shape()
        num_in = int(shape[1])
        W = tf.get_variable(
            name="W", shape=[num_in, num_out], initializer=initializer)
        b = tf.get_variable(
            name="b", shape=[num_out], initializer=tf.constant_initializer(0))

        out = tf.matmul(inp, W) + b
        return out


def fclayer3d(inp, num_out, act=tf.nn.tanh, name=None, reuse=None):
    if name is None:
        name = "fclinear3d"
    with tf.variable_scope(name, reuse=reuse):
        shape = inp.get_shape()
        d = int(shape[2])
        l = int(shape[1])
        inp = tf.reshape(inp, [-1, d])
        out = fclinear(inp=inp, num_out=num_out, act=act)
        out = tf.reshape(out, [-1, l, num_out])
        return out

In [ ]:
# Defining the input variables
input_seq = tf.placeholder(
            dtype=tf.int32,
            shape=[None, max_len],
            name="input_seq"
        )  # [x, l]

seq_len = tf.placeholder(
            dtype=tf.int32,
            shape=[None],
            name="seq_len"
        )  # [x]

labels = tf.placeholder(
            dtype=tf.int32,
            shape=[None, 26],
            name="labels"
        )  # [x, 26]

# One hot encoding of the input
one_hot_seq = tf.one_hot(input_seq, depth=26)  # [x, l, 26]

# Creating the LSTM

In [ ]:
lstm_inp = fclayer3d(one_hot_seq, lstm_dim)  # [x, l, lstm_dim]

cell_fw = tf.nn.rnn_cell.LSTMCell(
            lstm_dim,
            initializer=initializer,
            state_is_tuple=True,
            activation=tf.nn.tanh
        )

cell_bw = tf.nn.rnn_cell.LSTMCell(
            lstm_dim,
            initializer=initializer,
            state_is_tuple=True,
            activation=tf.nn.tanh
        )

lstm_inp = tf.transpose(lstm_inp, [1, 0, 2])
lstm_inp = tf.unpack(lstm_inp)

outputs, state_fw, state_bw = tf.nn.bidirectional_rnn(
            cell_fw=cell_fw,
            cell_bw=cell_bw,
            inputs=lstm_inp,
            dtype=tf.float32,
            sequence_length=seq_len
        )

output_state = tf.pack(state_fw, state_bw)
output_state = tf.transpose(output_state, [1, 0, 2])  # [x, 4, d]
output_state = tf.reshape(output_state, [-1, 4 * lstm_dim])  # [x, 4*d]

# Using LSTM output to predict

In [ ]:
pred_layer1 = fclayer(output_state, 500, name="pred1")
pred_layer2 = fclayer(pred_layer1, 500, name="pred2")
pred_out = fclayer(pred_layer2, 26, name="pred_out")  # [x, 26]
pred_probs = tf.nn.softmax(pred_out)

# Creating loss function and optimizer

In [ ]:
log_prob = tf.log(pred_probs)  # [x, 26]
loss = - log_prob * labels  # [x, 26]
loss = tf.reduce_sum(loss, axis=1)  # [x]
loss = tf.reduce_mean(loss)  # scalar

optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
init = tf.initialize_all_variables()
saver = tf.train.Saver()

# Train

In [1]:
def get_batches(data):
    num_batches = len(data) / batch_size
    for i in range(num_batches):
        batch_x = data[batch_size * i: batch_size * (i+1)]
        len_x = np.zeros(batch_size)
        batch_y = np.zeros((batch_size, 26))
        
        # TODO for each example in batch_x:
        # - Map each character to a number between 0 to 25.
        # - Randomly replace some numbers with -1 (-1 denotes '_'). Note: replace all occurences of the numbers.
        # - For each replaced number i, set batch_y[example, i] = number of occurrences of i.
        # - Compute the length of each example and store in the len_x.
        # - Append '-1' to all examples so that their length is equal to max_len
        
        yield batch_x, len_x, batch_y

In [11]:
for batch_x, len_x, batch_y in get_batches(train):
    print batch_x
    print len_x
    print batch_y
    break  # breaking early for testing

['reverentially', 'legends', 'firebrand', 'guardedness', 'ravishes', 'interred', 'dividers', 'droopy', 'disagreed', 'eats', 'installation', 'thankful', 'hungered', 'suckling', 'fiddling', 'fanfare', 'torture', 'bylaw', 'shoemaker', 'grey', 'detente', 'jagged', 'quartic', 'paedophiles', 'wielding', 'accentuate', 'inasmuch', 'transporters', 'floored', 'cleanses', 'depends', 'placement', 'casework', 'oxidiser', 'kickback', 'pardonable', 'steerable', 'godhead', 'hadron', 'fenland', 'pervade', 'lifters', 'pesetas', 'uncreative', 'stealers', 'gelding', 'rebury', 'proud', 'thorax', 'dishonestly', 'perverts', 'unconcernedly', 'knead', 'sicilian', 'hydro', 'undulate', 'stories', 'deceiving', 'tiny', 'headlights', 'nostril', 'phenotypes', 'essentially', 'skinhead']
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0

In [ ]:
with tf.Session as sess:
    if load_model:
        saver.restore(sess, model_weight_file)
        # If loading trained model, no need to train again
    else:
        sess.run(init)
        # TODO:
        # - Iterate over the batches and train the model
        # - Calculate the train and test accuracy after regular intervals to track your training
        # - Save your model after regular intervals

In [ ]:
def guesser(mask, guessed):
    # TODO:
    # - Define your guesser function so that it uses the trained model to guess characters.
    # - Return the character with the highest probability (or highest logit) and which has not been guessed yet.
    # - To get the logits from the model, you have to do the same kind of one-hot mapping as done in
    #   get_batches function.

# To convert to python script use:
 jupyter nbconvert --to script filename.ipynb